In [1]:
from math import *

def quaternion2rpy(w, x, y, z, form = 'Deg'):
	roll = atan2((-2 * y * z + 2 * x * w), (1 - 2 * x * x - 2 * y * y))
	pitch = asin(-2 * x * z + 2 * w * y)
	yaw = atan2((2 * x * y + 2 * w * z), (-2 * y * y - 2 * z * z + 1))
	if form == 'Deg':
		roll = degrees(roll)
		pitch = degrees(pitch)
		yaw = degrees(yaw)
	if yaw<0: yaw = 360 + yaw
	return {'roll': roll, 'pitch': pitch, 'yaw': yaw}

def rpy2quaternion(roll, pitch, yaw, form = 'Deg'):
	if form == 'Deg':
		roll = radians(roll) / 2
		pitch = radians(pitch) / 2
		yaw = radians(yaw) / 2
	elif form == 'Rad':
		roll = roll / 2
		pitch = pitch / 2
		yaw = yaw / 2
	w = cos(roll) * cos(pitch) * cos(yaw) + sin(roll) * sin(pitch) * sin(yaw)
	x = sin(roll) * cos(pitch) * cos(yaw) - cos(roll) * sin(pitch) * sin(yaw)
	y = cos(roll) * sin(pitch) * cos(yaw) + sin(roll) * cos(pitch) * sin(yaw)
	z = cos(roll) * cos(pitch) * sin(yaw) - sin(roll) * sin(pitch) * cos(yaw)
	return {'w': w, 'y': y, 'z': z, 'x': x}
'''
for i in range(360):
	_degree = i / 2
	sin_degree = sin(radians(_degree))
	cos_degree = cos(radians(_degree))
	dic = rpy2quaternion(0,0,i)
	D_A = quaternion2rpy(cos_degree, 0 ,0 , sin_degree)
	F_A = quaternion2rpy(dic['w'], dic['x'], dic['y'], dic['z'])
	print "Angle: %s Direct : z: %s w: %s  D_A: %s Func: z: %s w: %s F_A: %s" % (i, sin_degree, cos_degree, D_A['yaw'], dic['z'],dic['w'], F_A['yaw'])
'''

'\nfor i in range(360):\n\t_degree = i / 2\n\tsin_degree = sin(radians(_degree))\n\tcos_degree = cos(radians(_degree))\n\tdic = rpy2quaternion(0,0,i)\n\tD_A = quaternion2rpy(cos_degree, 0 ,0 , sin_degree)\n\tF_A = quaternion2rpy(dic[\'w\'], dic[\'x\'], dic[\'y\'], dic[\'z\'])\n\tprint "Angle: %s Direct : z: %s w: %s  D_A: %s Func: z: %s w: %s F_A: %s" % (i, sin_degree, cos_degree, D_A[\'yaw\'], dic[\'z\'],dic[\'w\'], F_A[\'yaw\'])\n'

In [2]:
%matplotlib inline

import os,sys
import os.path as osp

import argparse
import os, tempfile, glob, shutil
import random
from datetime import datetime

import GPy
import GPyOpt
import subprocess as sub
import numpy as np
import scipy.spatial
import math
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy import optimize
import time
import socket
from sklearn.neighbors import NearestNeighbors

global g_blender_executable_path,  blank_file, render_code, model_file, table_file, temp_dirname, epsilon, \
       filename, location_gt, rotation_gt, obj_start_config_array, hand_start_config_array, \
       obj_end_config_array, hand_end_config_array, id_of_sample, train_file, init_pose_file, final_pose_file

epsilon = 1e-6


In [3]:

if os.environ.get('BLENDER_PATH') == None:
    print("Please set BLENDER_PATH in bashrc!")
    sys.exit()
g_blender_executable_path = os.environ['BLENDER_PATH']

if os.environ.get('PHYSICS_SIMULATOR_PATH') == None:
    print("Please set PHYSICS_SIMULATOR_PATH in bashrc!")
    sys.exit()

g_repo_path = os.environ['PHYSICS_SIMULATOR_PATH']

# Input Pose File path
init_pose_file = g_repo_path + '/tmp/init_pose.txt'
final_pose_file = g_repo_path + '/tmp/final_pose.txt'

g_blank_blend_file_path = g_repo_path + '/blank.blend'
obj_model_folder = g_repo_path+'/obj_models/'
table_model_folder = g_repo_path+'/table_models/'
model_name = 'expo_dry_erase_board_eraser'
robot_name = 'reflex'
table_name = 'table'


blank_file = osp.join(g_blank_blend_file_path)
render_code = g_repo_path + '/render_gpyopt_dynamics_reflex_expo_socket.py'
model_file = osp.join(obj_model_folder, model_name+ '.obj')
robot_file = osp.join(obj_model_folder, robot_name+ '.obj')
table_file = osp.join(table_model_folder, table_name+ '.obj')
temp_dirname = '/home/zhusj/repos/blender_scene_generator/test'  #tempfile.mkdtemp()

In [4]:
# Peter KT Yu, Aug 2015
# Plot the trajectory

import numpy as np
import json

import matplotlib
# import matplotlib.patches as mpatches
# import matplotlib.pyplot as plt
# from matplotlib.collections import PatchCollection

# from config.shape_db import ShapeDB

import tf.transformations as tfm
from ik.helper import *
from config.helper import *
# from matplotlib.pyplot import savefig
import time

# from mpl_toolkits.mplot3d import Axes3D

import pdb
def extract2d_and_cleanup(data):

    # print "data: ", data
    tip_pose = data['tip_pose']
    object_pose = data['object_pose']

#     print "obj pose: ", object_pose[0]
    ft = data['ft_wrench']
    
    # transformation to the first 
    invT0 = np.linalg.inv(matrix_from_xyzquat(object_pose[0][1:4], object_pose[0][4:8]))
    sub = 1
    
    # object
    object_pose_2d = []
    for i in (range(0, len(object_pose), sub)):
        T = matrix_from_xyzquat(object_pose[i][1:4], object_pose[i][4:8])
        tip_pose_object0 = np.dot(invT0, T)
        scale, shear, angles, trans, persp = tfm.decompose_matrix(tip_pose_object0)
        #print 'trans', trans[0:2], 'angle', angles[2]
        time = object_pose[i][0]
        # don't add redundant data entry with the same time
        if(not(len(object_pose_2d) > 0 and time == object_pose_2d[-1][0] )):
            object_pose_2d.append([time] + trans[0:2].tolist() + [angles[2]])
    
    # probe
    tip_pose_2d = []
    for i in (range(0, len(tip_pose), sub)):
        tip_pose_0 = np.dot(invT0, tip_pose[i][1:4]+[1])
        #print 'trans', tip_pose_0[0:2]
        time = tip_pose[i][0]
        
        # don't add redundant data entry with the same time
        if(not(len(tip_pose_2d) > 0 and time == tip_pose_2d[-1][0] )):
            tip_pose_2d.append([time] + tip_pose_0[0:2].tolist())

    # ft, no redundency
    ft_2d = np.array(ft)[:,0:3].tolist()   # only need the force
#     print 'object_pose_2d', len(object_pose_2d), 'tip_pose_2d', len(tip_pose_2d), 'ft_2d', len(ft_2d)
    print 'object_pose_2d', len(object_pose_2d[0]), 'tip_pose_2d', len(tip_pose_0), 'ft_2d', len(ft_2d)

    data2d = {}
    data2d['tip_poses_2d'] = tip_pose_2d
    data2d['object_poses_2d'] = object_pose_2d
    data2d['force_2d'] = ft_2d
    return data2d


import pandas as pd

def resample_using_pandas(data):
    force_2d = data['force_2d']
    object_poses_2d = data['object_poses_2d']
    tip_poses_2d = data['tip_poses_2d']
    starttime = max(tip_poses_2d[0][0], object_poses_2d[0][0], force_2d[0][0])
    endtime = min(tip_poses_2d[-1][0], object_poses_2d[-1][0], force_2d[-1][0])
    
    pd_starttime = pd.to_datetime(starttime, unit='s')
    pd_endtime = pd.to_datetime(endtime, unit='s')
    
    tip_poses_2d_dt = pd.to_datetime(np.array(tip_poses_2d)[:,0].tolist(), unit='s')    
    tip_poses_2d = pd.DataFrame(np.array(tip_poses_2d)[:,1:3].tolist(), index=tip_poses_2d_dt)
    tip_poses_2d_resampled = tip_poses_2d.resample('10ms', how='mean')
    tip_poses_2d_interp = tip_poses_2d_resampled.interpolate()
    
    start_ = tip_poses_2d_interp.index.searchsorted(pd_starttime)
    end_ = tip_poses_2d_interp.index.searchsorted(pd_endtime)
    tip_poses_2d_interp = tip_poses_2d_interp.ix[start_:end_]
    tip_poses_2d_interp_list = tip_poses_2d_interp.values.tolist()
    
    object_poses_2d_dt = pd.to_datetime(np.array(object_poses_2d)[:,0].tolist(), unit='s')
    object_poses_2d = pd.DataFrame(np.array(object_poses_2d)[:,1:4].tolist(), index=object_poses_2d_dt)
    object_poses_2d_resampled = object_poses_2d.resample('10ms', how='mean')
    object_poses_2d_interp = object_poses_2d_resampled.interpolate()
    start_ = object_poses_2d_interp.index.searchsorted(pd_starttime)
    end_ = object_poses_2d_interp.index.searchsorted(pd_endtime)
    object_poses_2d_interp = object_poses_2d_interp.ix[start_:end_]
    object_poses_2d_interp_list = object_poses_2d_interp.values.tolist()
    
    force_dt = pd.to_datetime(np.array(force_2d)[:,0].tolist(), unit='s')
    force_2d = pd.DataFrame(np.array(force_2d)[:,1:3].tolist(), index=force_dt)
    force_2d_resampled = force_2d.resample('10ms', how='mean')
    force_2d_interp = force_2d_resampled.interpolate()
    start_ = force_2d_interp.index.searchsorted(pd_starttime)
    end_ = force_2d_interp.index.searchsorted(pd_endtime)
    force_2d_interp = force_2d_interp.ix[start_:end_]
    force_2d_interp_list = force_2d_interp.values.tolist()
    
    
    data_resample = {}
    data_resample['tip_poses_2d'] = tip_poses_2d_interp_list
    data_resample['object_poses_2d'] = object_poses_2d_interp_list
    data_resample['force_2d'] = force_2d_interp_list
    return data_resample
    
def animate_2dsynced(data, shape_id, figfname):
    fig, ax = plt.subplots()
    probe_radius = 0.004745   # probe1: 0.00626/2 probe2: 0.004745
    
    v = int(figfname.split('_')[-4].split('=')[1])
    sub = 1                 # subsample rate
    tip_pose = data['tip_poses_2d']
    object_pose = data['object_poses_2d']
    force = data['force_2d']
    
    patches = []
    
    
    # add the object as polygon
    shape_db = ShapeDB()
    shape_polygon = shape_db.shape_db[shape_id]['shape_poly'] # shape of the objects presented as polygon.
    shape_polygon_3d = np.hstack((np.array(shape_polygon), np.zeros((len(shape_polygon), 1)), np.ones((len(shape_polygon), 1))))
    

    print 'object_pose', len(object_pose), 'tip_pose', len(tip_pose), 'force', len(force)
    plt.ion()
    for i in (range(0, len(tip_pose), sub)):
        
        plt.cla()
        T = tfm.compose_matrix(translate = object_pose[i][0:2] + [0], angles = (0,0,object_pose[i][2]) )
        shape_polygon_3d_world = np.dot(T, shape_polygon_3d.T)
        
        obj = mpatches.Polygon(shape_polygon_3d_world.T[:,0:2], closed=True, color='blue', alpha=0.05)
        ax.add_patch(obj)
    
        # add the probes as circle
        circle = mpatches.Circle(tip_pose[i][0:2], probe_radius, ec="none", color='red', alpha=0.5)
        ax.add_patch(circle)
        
        # add the force
        ax.arrow(tip_pose[i][0], tip_pose[i][1], force[i][0]/100, force[i][1]/100, head_width=0.005, head_length=0.01, fc='k', ec='k')
        
        #arrow = mpatches.Arrow(tip_pose[i][0], tip_pose[i][1], force[i][0],
        #                force[i][1], head_width=0.05, head_length=0.1, fc='k', ec='k')
        #ax.add_patch(arrow)
        
        # render it
        plt.axis([-0.1, 0.1, -0.1, 0.1])
        #plt.axis('equal')
        plt.draw()
        #time.sleep(0.1)
    plt.show()

def wraptopi(data):
    return ( data + np.pi) % (2 * np.pi ) - np.pi 

def extend_with_velocity(data, h):
    h = 0.01  # from 10ms
    
    tip_pos = np.array(data['tip_poses_2d'])
    object_pos = np.array(data['object_poses_2d'])
    #nd = tip_pose.shape[0]
    
    tip_vel = (tip_pos[2:,:] - tip_pos[0:-2,:]) / 2 / h
    tip_pos = tip_pos[1:-1,:]
    
    object_vel_xy = (object_pos[2:,0:2] - object_pos[0,0:2]) / 2 / h
    object_vel_th = wraptopi(object_pos[2:,2:] - object_pos[0,2:]) / 2 / h
    object_pos = object_pos[1:-1,:]
    
    data_extended = {}
    data_extended['tip_poses_2d'] = tip_pos.tolist()
    data_extended['tip_vels_2d_vel'] = tip_vel.tolist()
    data_extended['object_poses_2d'] = object_pos.tolist()
    data_extended['object_vels_2d'] = np.hstack((object_vel_xy, object_vel_th)).tolist()
    data_extended['force_2d'] = data['force_2d'][1:-1]
    
    return data_extended


# data: synced 2d data
def extract_training_data(data):    
    tip_pose = data['tip_poses_2d']
    tip_vel = data['tip_vels_2d_vel']
    object_pose = data['object_poses_2d']
    object_vel = data['object_vels_2d']
    force = data['force_2d']
    
    
    # find the beginning of contact by force magnitude
    threshold = 0.5
    start_index = 0
    end_index = 0
    sub = 5
    for i in (range(0, len(force)-1, 1)):
        if norm(np.array(tip_pose[i]) - np.array(tip_pose[i+1]))>0 and norm(np.array(object_pose[i]) - np.array(object_pose[i+1]))>1e-3 and norm(force[i]) > threshold:
            start_index = i
            break
            
    for i in (range(len(force)-2, -1, -1)):
        if norm(np.array(tip_pose[i]) - np.array(tip_pose[i+1]))>0 and norm(np.array(object_pose[i]) - np.array(object_pose[i+1]))>1e-3 and norm(force[i]) > threshold:
            end_index = i
            break
#     print 'start_index', start_index, 'end_index', end_index, 'len', len(force)
    
    data_training = []  
    labels = ['$x$', '$y$', '$\Delta x$', '$\Delta y$', 'force $x$', 'force $y$', r'$\Delta x$', r'$\Delta y$', r'$\Delta \theta$',
        '$v_x$', '$v_y$', '$v_x$', '$v_y$',  # 9:  tip_svx, tip_svy, tip_evx, tip_evy, 
        '$v_x$', '$v_y$', r'$\omega$',       # 13: object_pose_svx, object_pose_svy, object_pose_svtheta, # start speed
        '$v_x$', '$v_y$', r'$\omega$']       # 16: object_pose_evx, object_pose_evy, object_pose_evtheta  # end speed]
                                             # 19: tip_speedset (mm)
    
    for i in (range(start_index, end_index+1-sub, sub)):
        tip_pose_i_obji = transform_to_frame2d(tip_pose[i], object_pose[i])
        tip_pose_isub_obji = transform_to_frame2d(tip_pose[i+sub], object_pose[i])
        
        tip_vel_i_obji = rotate_to_frame2d(tip_vel[i], object_pose[i])
        tip_vel_isub_obji = rotate_to_frame2d(tip_vel[i+sub], object_pose[i])
        
        force_i_obji = rotate_to_frame2d(force[i], object_pose[i])
        
        object_pose_i_obji = transform_to_frame2d(object_pose[i][0:2], object_pose[i])
        object_pose_isub_obji = transform_to_frame2d(object_pose[i+sub][0:2], object_pose[i])
        
        object_vel_i_obji = rotate_to_frame2d(object_vel[i][0:2], object_vel[i])
        object_vel_isub_obji = rotate_to_frame2d(object_vel[i+sub][0:2], object_vel[i])
        
        newdata = tip_pose_i_obji + (np.array(tip_pose_isub_obji) - np.array(tip_pose_i_obji)).tolist() +\
        force_i_obji + (np.array(object_pose_isub_obji) - np.array(object_pose_i_obji)).tolist() + \
        [object_pose[i+sub][2] - object_pose[i][2]] + \
        tip_vel_i_obji + tip_vel_isub_obji + \
        object_vel_i_obji + [object_vel[i][2]] + object_vel_isub_obji + [object_vel[i+sub][2]]
        
        #print newdata
        data_training.append(newdata)
        
    return data_training

def extend_with_speedset(data, v):
    return [d + [v] for d in data]

def json2trainingdata(filepath):
        
    with open(filepath) as data_file:    
        data = json.load(data_file)
    
    shape_id = 'rect1'
    v = getfield_from_filename(filepath, 'v')
    data2d = extract2d_and_cleanup(data)
    data_synced = resample_using_pandas(data2d)
    #animate_2dsynced(data_synced, shape_id, filepath.replace('.json', '.png'))
    data_synced = extend_with_velocity(data_synced, h=0.01)
    
    data_training = extract_training_data(data_synced)
    data_training = extend_with_speedset(data_training, int(v))
    return data_synced
#     return data_training




    


In [45]:
import glob

path = '/home/zhusj/data/push/abs/rect1/rect1_json'

filelist = glob.glob("%s/motion*.json" % path)

all_training_data = []
no_of_samples = 100



for json_filepath in filelist:
    # only process v=20
    # if json_filepath.find('v=20_') == -1:
        # continue

    # try:
    if len(all_training_data) < no_of_samples:
#     print 'json_filepath', json_filepath

        data_synced = json2trainingdata(json_filepath)
        all_training_data.append(data_synced)
#         all_training_data.extend(json2trainingdata(json_filepath))
#         print 'len(all_training_data)', len(all_training_data)
    else:
        break
    # except:
        # print json_filepath
        # pass
#     print len(data_synced['object_poses_2d'])
#     print 'data_synced', data_synced
#     print 'data_training', data_training

    
#     X_obj = data_synced['object_poses_2d'][0]
#     Y_obj = data_synced['object_poses_2d'][-1]
    
#     X_tip = data_synced['tip_poses_2d'][0]
#     Y_tip = data_synced['tip_poses_2d'][-1]
    
#     t = len(data_synced['object_poses_2d'])*0.01
    
    

#     print X_obj, Y_obj, X_tip, Y_tip, t
#     raw_input("Press Enter to continue...")


# outputfile= "%s/data_training_with_vel.json" % argv[1]
# with open(outputfile, 'w') as outfile:
#     json.dump(all_training_data, outfile, indent=4)
no_of_training_samples = 20

samples = np.random.choice(len(all_training_data), no_of_samples, replace=False)
train_id = samples[:no_of_training_samples]
test_id = samples[no_of_training_samples:]

train_samples = []
test_samples = []
for i,val in enumerate(train_id):
    train_samples.append(all_training_data[val])
for i,val in enumerate(test_id):
    test_samples.append(all_training_data[val])

object_pose_2d 4 tip_pose_2d 4 ft_2d 175
object_pose_2d 4 tip_pose_2d 4 ft_2d 172
object_pose_2d 4 tip_pose_2d 4 ft_2d 186
object_pose_2d 4 tip_pose_2d 4 ft_2d 191
object_pose_2d 4 tip_pose_2d 4 ft_2d 196
object_pose_2d 4 tip_pose_2d 4 ft_2d 182
object_pose_2d 4 tip_pose_2d 4 ft_2d 172
object_pose_2d 4 tip_pose_2d 4 ft_2d 200
object_pose_2d 4 tip_pose_2d 4 ft_2d 202
object_pose_2d 4 tip_pose_2d 4 ft_2d 182
object_pose_2d 4 tip_pose_2d 4 ft_2d 176
object_pose_2d 4 tip_pose_2d 4 ft_2d 187
object_pose_2d 4 tip_pose_2d 4 ft_2d 182
object_pose_2d 4 tip_pose_2d 4 ft_2d 197
object_pose_2d 4 tip_pose_2d 4 ft_2d 171
object_pose_2d 4 tip_pose_2d 4 ft_2d 185
object_pose_2d 4 tip_pose_2d 4 ft_2d 171
object_pose_2d 4 tip_pose_2d 4 ft_2d 190
object_pose_2d 4 tip_pose_2d 4 ft_2d 189
object_pose_2d 4 tip_pose_2d 4 ft_2d 188
object_pose_2d 4 tip_pose_2d 4 ft_2d 177
object_pose_2d 4 tip_pose_2d 4 ft_2d 184
object_pose_2d 4 tip_pose_2d 4 ft_2d 175
object_pose_2d 4 tip_pose_2d 4 ft_2d 173
object_pose_2d 4

In [7]:
# def test( mass, friction):
#     print "testing.."
#     y_test = []
#     y_new = 0

#     for count in xrange(0, len(test_samples)): 

#         data = test_samples[count]

#         X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
        
# #         print 'input: ', count, X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
# #                          x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
        
# #         raw_input("Press Enter to continue...")
   
#         with open(init_pose_file, "w+") as file:
#             file.write("%f %f %f %f %f %f %f %f %f %f %f %f %f \n" % \
#                        (mass, friction, X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
#                         x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t))
        
        
#         # Create a TCP/IP socket
#         sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

#         # Connect the socket to the port where the server is listening
#         serv_address = ('localhost', 50007)
# #         print('waiting for a connection')
#         sock2.connect(serv_address)
# #         print('Connected')
#         sock2.sendall(str(count))
#         sock2.close()   

#         sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#         sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
#         # Bind the socket to the port
#         server_address = ('localhost', 10000)
#         sock.bind(server_address)

#         # Listen for incoming connections
#         sock.listen(1)        
#         connection, client_address = sock.accept()

#         # Receive the data in small chunks and retransmit it
#         data = connection.recv(16)
#         connection.close()
#         sock.close()
        
        
#         Y_new = np.zeros((1, 7))

#         with open(final_pose_file, "r") as file:
#             for line in file:
#                 vals = line.split()
#                 for i in xrange(0,7):
#                     Y_new[0,i] = vals[i]
# #         print Y_new
# #         location, rotation = run_physics_engine(train_file, count, mass, friction, force_gt)  
        
        
#         location = Y_new[0,0:2]
#         location_gt = np.array(Y_obj[0:2])
#         rotation = Y_new[0,3:]
#         rotation_gt = np.array([y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])

        
# #         print 'location_gt: ', location_gt, rotation_gt
            
#         dist = np.sqrt(np.sum(((location-location_gt)**2)))
# #         dist = np.sqrt(np.sum(((np.array(Y_new[0:2])-np.array(Y_obj[0:2]))**2)))
#         q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        


# #         print dist, q_dist
# #         y_new += dist + q_dist


# #         print dist, q_dist
#         y_test.append(dist + q_dist)
# #         print y_test
#         y_new += y_test[count]
#         count += 1

# #         print "y_new", y_new

#     return y_new, y_test

In [31]:
def test( mass, friction):
    print "testing.."
    y_test = []
    y_new = 0
    p_error = 0 
    r_error = 0
    angle_error = 0
    for count in xrange(0, len(test_samples)): 

        data = test_samples[count]

        X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
        
#         print 'input: ', count, X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
#                          x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
        
#         raw_input("Press Enter to continue...")
   
        with open(init_pose_file, "w+") as file:
            file.write("%f %f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                       (mass, friction, X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                        x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t))
        
        
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50007)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(count))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10000)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, 7))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for i in xrange(0,7):
                    Y_new[0,i] = vals[i]
#         print Y_new
#         location, rotation = run_physics_engine(train_file, count, mass, friction, force_gt)  
        
        
        location = Y_new[0,0:2]
        location_gt = np.array(Y_obj[0:2])
        rotation = Y_new[0,3:]
        rotation_gt = np.array([y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])
        
        angles = quaternion2rpy(Y_new[0,3], Y_new[0,4], Y_new[0,5], Y_new[0,6], form = 'Deg')
        angles_gt = quaternion2rpy(y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z'], form = 'Deg')

#         print 'angles: ', angles, angles_gt
            
        dist = np.sqrt(np.sum(((location-location_gt)**2)))
#         dist = np.sqrt(np.sum(((np.array(Y_new[0:2])-np.array(Y_obj[0:2]))**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        

        if angles['yaw'] > 180:
            angles['yaw'] -= 360
        if angles_gt['yaw'] > 180:
            angles_gt['yaw'] -= 360 
#         print 'angles: ', angles, angles_gt

        angle_dist = np.abs(angles['yaw'] - angles_gt['yaw'])

#         print dist, q_dist
#         y_new += dist + q_dist


#         print dist, q_dist
        y_test.append(dist + q_dist)
#         print y_test
        y_new += y_test[count]
        count += 1
        p_error += dist
        r_error += q_dist
        angle_error += angle_dist

#         print "y_new", y_new

    return y_new, y_test, p_error, r_error, angle_error

In [6]:
def transform_data(data):

    X_obj = data['object_poses_2d'][0]
    Y_obj = data['object_poses_2d'][-1]
    
    X_traj
    
    x_quat = rpy2quaternion(0, 0, X_obj[2], form = 'Rad')
    y_quat = rpy2quaternion(0, 0, Y_obj[2], form = 'Rad')

    X_tip = data['tip_poses_2d'][0]
    Y_tip = data['tip_poses_2d'][-1]

    t = len(data_synced['object_poses_2d'])*0.01
        
    return X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t    

In [48]:
def get_current_train_loss(mass, friction):

    print 'mass, friction: ', mass, friction, myBopt.Y_best
    p_error = 0
    r_error = 0
    angle_error = 0
    
    y_new = 0
#     count = 0
#     if new_iter is True:
# #   randomly select (number_of_samples) samples
#         samples = random.sample(xrange(0, len(train_samples)), number_of_samples)
#         new_iter = False 
    
#     for count in samples:  
    for count in xrange(0, number_of_samples):
#         print "input: ", train_file, count, mass, friction, force_gt,count

        data = train_samples[count]

        X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
        
#         print 'input: ', count, X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
#                          x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
        
#         raw_input("Press Enter to continue...")
   
        with open(init_pose_file, "w+") as file:
            file.write("%f %f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                       (mass, friction, X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                        x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t))
        
        
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50007)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(count))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10000)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, 7))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for i in xrange(0,7):
                    Y_new[0,i] = vals[i]
#         print Y_new
#         location, rotation = run_physics_engine(train_file, count, mass, friction, force_gt)  
        
        
        location = Y_new[0,0:2]
        location_gt = np.array(Y_obj[0:2])
        rotation = Y_new[0,3:]
        rotation_gt = np.array([y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])

        angles = quaternion2rpy(Y_new[0,3], Y_new[0,4], Y_new[0,5], Y_new[0,6], form = 'Deg')
        angles_gt = quaternion2rpy(y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z'], form = 'Deg')
        
#         print 'location_gt: ', location_gt, rotation_gt
            
        dist = np.sqrt(np.sum(((location-location_gt)**2)))
#         dist = np.sqrt(np.sum(((np.array(Y_new[0:2])-np.array(Y_obj[0:2]))**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        
    
        
        if angles['yaw'] > 180:
            angles['yaw'] -= 360
        if angles_gt['yaw'] > 180:
            angles_gt['yaw'] -= 360 
#         print 'angles: ', angles, angles_gt

        angle_dist = np.abs(angles['yaw'] - angles_gt['yaw'])

        y_new += y_test[count]
        count += 1
        p_error += dist
        r_error += q_dist
        angle_error += angle_dist

    pos_cost.append(p_error/number_of_samples)
    angle_cost.append(angle_error/number_of_samples)

#     print "X, y_new",X,y_new
#     raw_input("Press Enter to continue...")

    return y_new


In [68]:
 train_samples[0]

{'force_2d': [[0.029111769999999426, -0.041389469999999984],
  [0.02145510333333271, 0.0020571966666666683],
  [0.0026432699999994202, 0.017931030000000014],
  [0.029873103333332818, -0.031068136666666673],
  [0.046248769999999384, -0.013184970000000018],
  [0.06498676999999937, -0.019260470000000012],
  [0.050018769999999435, -0.020524969999999976],
  [0.05124776999999936, 0.05524586333333333],
  [0.051620269999999246, 0.00427403000000004],
  [0.05414243666666607, -0.004470470000000004],
  [0.024891269999999466, 0.03811102999999999],
  [0.02674910333333284, -0.020268803333333325],
  [0.019272269999999425, 0.009180030000000006],
  [0.0069191033333327505, 0.017922196666666685],
  [0.01451076999999934, -0.03165796999999998],
  [0.018502103333332798, -0.011095469999999996],
  [-0.07576273000000056, -0.22331447000000004],
  [-0.16377973000000057, -0.47983597],
  [-0.48598723000000055, -1.1256138033333332],
  [-0.7706382300000005, -1.5019669699999998],
  [-0.9733455633333339, -1.83508713666

In [82]:
def myf(X):

#     y_current_loss = get_current_train_loss(myBopt.x_opt[0], myBopt.x_opt[1])
     
    mass = X[0][0]
    friction = X[0][1]
    
    y_new = 0
#     count = 0
#     if new_iter is True:
# #   randomly select (number_of_samples) samples
#         samples = random.sample(xrange(0, len(train_samples)), number_of_samples)
#         new_iter = False 
    
#     for count in samples:  
    for count in xrange(0, number_of_samples):
#         print "input: ",count, mass, friction, force_gt,count

        data = train_samples[count]

        X_obj, x_quat, Y_obj, y_quat, X_tip, Y_tip, t = transform_data(data)
        
#         print 'input: ', count, X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
#                          x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
        
#         raw_input("Press Enter to continue...")
   
        with open(init_pose_file, "w+") as file:
            file.write("%f %f %f %f %f %f %f %f %f %f %f %f %f \n" % \
                       (mass, friction, X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                        x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t))
        
        
        # Create a TCP/IP socket
        sock2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock2.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

        # Connect the socket to the port where the server is listening
        serv_address = ('localhost', 50007)
#         print('waiting for a connection')
        sock2.connect(serv_address)
#         print('Connected')
        sock2.sendall(str(count))
        sock2.close()   

        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        # Bind the socket to the port
        server_address = ('localhost', 10000)
        sock.bind(server_address)

        # Listen for incoming connections
        sock.listen(1)        
        connection, client_address = sock.accept()

        # Receive the data in small chunks and retransmit it
        data = connection.recv(16)
        connection.close()
        sock.close()
        
        
        Y_new = np.zeros((1, 7))

        with open(final_pose_file, "r") as file:
            for line in file:
                vals = line.split()
                for i in xrange(0,7):
                    Y_new[0,i] = vals[i]
#         print Y_new
#         location, rotation = run_physics_engine(train_file, count, mass, friction, force_gt)  
        
        
        location = Y_new[0,0:2]
        location_gt = np.array(Y_obj[0:2])
        rotation = Y_new[0,3:]
        rotation_gt = np.array([y_quat['w'], y_quat['x'], y_quat['y'], y_quat['z']])

        
#         print 'location_gt: ', location_gt, rotation_gt
            
        dist = np.sqrt(np.sum(((location-location_gt)**2)))
#         dist = np.sqrt(np.sum(((np.array(Y_new[0:2])-np.array(Y_obj[0:2]))**2)))
        q_dist = 1 - np.sum(np.multiply(rotation, rotation_gt)**2)        


#         print 'dist:', dist, q_dist
#         y_new += dist + q_dist
        y_new += q_dist

        count += 1
    y_new /= number_of_samples
    print "X, y_new",X,y_new
#     raw_input("Press Enter to continue...")

    return y_new


In [49]:
path = '/home/zhusj/data/push/abs/rect1/rect1_json'
filelist = glob.glob("%s/motion*.json" % path)
all_training_data = []

for json_filepath in filelist:
    data_synced = json2trainingdata(json_filepath)
    all_training_data.append(data_synced)

object_pose_2d 4 tip_pose_2d 4 ft_2d 165
object_pose_2d 4 tip_pose_2d 4 ft_2d 175
object_pose_2d 4 tip_pose_2d 4 ft_2d 386
object_pose_2d 4 tip_pose_2d 4 ft_2d 253
object_pose_2d 4 tip_pose_2d 4 ft_2d 218
object_pose_2d 4 tip_pose_2d 4 ft_2d 201
object_pose_2d 4 tip_pose_2d 4 ft_2d 202
object_pose_2d 4 tip_pose_2d 4 ft_2d 1377
object_pose_2d 4 tip_pose_2d 4 ft_2d 176
object_pose_2d 4 tip_pose_2d 4 ft_2d 306
object_pose_2d 4 tip_pose_2d 4 ft_2d 296
object_pose_2d 4 tip_pose_2d 4 ft_2d 292
object_pose_2d 4 tip_pose_2d 4 ft_2d 172
object_pose_2d 4 tip_pose_2d 4 ft_2d 250
object_pose_2d 4 tip_pose_2d 4 ft_2d 149
object_pose_2d 4 tip_pose_2d 4 ft_2d 263
object_pose_2d 4 tip_pose_2d 4 ft_2d 733
object_pose_2d 4 tip_pose_2d 4 ft_2d 183
object_pose_2d 4 tip_pose_2d 4 ft_2d 212
object_pose_2d 4 tip_pose_2d 4 ft_2d 292
object_pose_2d 4 tip_pose_2d 4 ft_2d 231
object_pose_2d 4 tip_pose_2d 4 ft_2d 189
object_pose_2d 4 tip_pose_2d 4 ft_2d 242
object_pose_2d 4 tip_pose_2d 4 ft_2d 193
object_pose_2d 

In [50]:
len(all_training_data)

7128

In [ ]:
import time
import dill
import pickle
import glob

# global new_iter

# new_iter = True

BO_result = []
BO_time =[]
BO_test_loss =[]

RS_result =[]
RS_time =[]
RS_test_loss =[]

from GPyOpt.acquisitions.EI import AcquisitionEI
from GPyOpt.acquisitions.MPI import AcquisitionMPI
force_gt = float(48)


#     import pdb; pdb.set_trace()

############################# GPyOpt setup ######################

from GPyOpt.acquisitions.EI import AcquisitionEI
from GPyOpt.acquisitions.MPI import AcquisitionMPI

space = GPyOpt.Design_space(space =[{'name': 'mass', 'type': 'continuous', 'domain': (0.5,1.0)},
                                    {'name': 'friction', 'type': 'continuous', 'domain': (0.001,0.3)}])
mass_min = 0.5
mass_max = 1.5
friction_max = 0.5
total_trial = 50


bounds = [{'name': 'mass', 'type': 'continuous', 'domain': (mass_min,mass_max)},
        {'name': 'friction', 'type': 'continuous', 'domain': (1e-9,friction_max)}]

model = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False, normalize_Y = False)

aquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(space)

from numpy.random import seed # fixed seed
seed(123456)
initial_design = GPyOpt.util.stats.initial_design('random', space, 2)

acquisition = AcquisitionMPI(model, space, optimizer=aquisition_optimizer) 

evaluator = GPyOpt.core.evaluators.Sequential(acquisition)

# objective = GPyOpt.core.task.SingleObjective(myf)
# number_of_samples = 1
# myBopt = GPyOpt.methods.ModularBayesianOptimization(model, space, objective, acquisition, evaluator, \
#                                                     initial_design)

unit_time = 1
unit_iter = 30
max_time = 0
max_iter = 0
eps = 1e-3


# path = '/home/zhusj/data/push/abs/rect1/rect1_json'
# filelist = glob.glob("%s/motion*.json" % path)
no_of_samples = 100
no_of_training_samples = 20

# all_training_data = []

# for json_filepath in filelist:
#     if len(all_training_data) < no_of_samples:
#         data_synced = json2trainingdata(json_filepath)
#         all_training_data.append(data_synced)
#     else:
#         break

number_of_samples = len(train_samples)
# for count in xrange(0,10):
for count in xrange(0,total_trial):

    ####################### READ DATA ###################

            
    samples = np.random.choice(len(all_training_data), no_of_samples, replace=False)
    train_id = samples[:no_of_training_samples]
    test_id = samples[no_of_training_samples:]

    train_samples = []
    test_samples = []
    for i,val in enumerate(train_id):
        train_samples.append(all_training_data[val])
    for i,val in enumerate(test_id):
        test_samples.append(all_training_data[val])

        
        ############################################
    print "Starting trial ", count, datetime.now()
    
    pos_cost = []
    angle_cost = []
    
    print "Using", number_of_samples, "training sample(s)"
#     random.seed(datetime.now())

#     myBopt.run_optimization(max_time=max_time)
    max_time = unit_time * number_of_samples
    max_iter = unit_iter * number_of_samples
#     eps /= 2
    
#     objective = GPyOpt.core.task.SingleObjective(myf)
#     myBopt = GPyOpt.methods.ModularBayesianOptimization(model, space, objective, acquisition, evaluator, \
#                                                     initial_design)
    k = GPy.kern.Matern32(input_dim=2, ARD=True)
    
    myBopt = GPyOpt.methods.BayesianOptimization(myf,
                                                domain=bounds,
                                                kernel = k)
    start_time = time.time()
#     myBopt.run_optimization(max_time=max_time)
#     myBopt = GPyOpt.methods.ModularBayesianOptimization(model, space, objective, acquisition, evaluator, \
#                                                     initial_design)
    myBopt.run_optimization(max_time=max_time, eps=eps)
    elapsed_time = time.time() - start_time

    print "optima!!!!!!!!!!!!!!!!: ", myBopt.x_opt, myBopt.fx_opt, " in ", elapsed_time, myBopt.cum_time
    
    y_new, y_test, p_error, r_error, angle_error = test(myBopt.x_opt[0], myBopt.x_opt[1])
    
    print "current loss:", p_error/len(test_samples), angle_error/len(test_samples)
    print "cost: ", pos_cost, angle_cost
    
  
    BO_test_loss.append([y_new, y_test, p_error/len(test_samples), angle_error/len(test_samples)])
    BO_result.append(myBopt)
    BO_time.append([myBopt.cum_time,elapsed_time])
    




################## Random Search #############

    X = np.zeros((1,2))
    random.seed(datetime.now())

    print "starting trial", count, datetime.now()
    best_y = 10000
    best_f = 1
    best_m = 1
    result = []
    
    start_time = time.time()
    elapsed_time = time.time() - start_time

    while elapsed_time < max_time:
        m = (mass_max-mass_min)*random.random() + mass_min
        f = friction_max*random.random()
        X[0][0] = m
        X[0][1] = f

        y = myf(X)
        elapsed_time = time.time() - start_time

        if y < best_y:
            best_y = y
            best_m = m
            best_f = f

        result.append([m,f,y,best_m,best_f,best_y])
    print "optima!!!!!!!!!!!!!!!!: ", best_m,best_f,best_y, elapsed_time
    
    y_new, y_test, p_error, r_error, angle_error = test(best_m, best_f)
    
    print "current loss:", p_error/len(test_samples), angle_error/len(test_samples)
    print "cost: ", pos_cost, angle_cost
    
    RS_test_loss.append([y_new, y_test, p_error/len(test_samples), angle_error/len(test_samples)])    
    RS_result.append(result)
    RS_time.append(elapsed_time)

now = datetime.now()

save_data_filename = 'BO_result_'+str(now)
with open(save_data_filename, 'wb') as f:
    dill.dump([BO_result,BO_test_loss,BO_time], f)

max_number = 0
for i in xrange(0,total_trial):  
#     print "current length:" , len(RS_result[i])
    if max_number <  len(BO_result[i].Y_best):
        max_number =  len(BO_result[i].Y_best)
        print "max_number:", max_number

data = np.empty([total_trial,max_number])
for i in xrange(0,total_trial):  
    print len(BO_result[i].Y_best)
    for j in xrange(0,len(BO_result[i].Y_best)):        
        data[i][j] = BO_result[i].Y_best[j]
    for j in xrange(len(BO_result[i].Y_best), max_number):
        data[i][j] = data[i][j-1]
dm = np.mean(data, axis=0)
ds = np.std(data, axis=0)    
    
now = datetime.now()
save_data_filename = 'RS_result_'+str(now)
with open(save_data_filename, 'wb') as f:
    pickle.dump([RS_result,RS_test_loss,RS_time], f)
    

max_number = 0
for i in xrange(0,total_trial):  
#     print "current length:" , len(RS_result[i])
    if max_number <  len(RS_result[i]):
        max_number =  len(RS_result[i])
        print "max_number:", max_number

data1 = np.empty([total_trial,max_number])
for i in xrange(0,total_trial):  
    print len(RS_result[i])
    for j in xrange(0,len(RS_result[i])):        
        data1[i][j] = RS_result[i][j][5]
    for j in xrange(len(RS_result[i]), max_number):
        data1[i][j] = data1[i][j-1]
        
dm1 = np.mean(data1, axis=0)
ds1 = np.std(data1, axis=0)

Starting trial  0 2017-01-27 14:46:52.986227
Using 20 training sample(s)
X, y_new [[ 0.78746143  0.05081275]] 0.197634105848
X, y_new [[ 0.92076827  0.4727981 ]] 0.00552816261485
X, y_new [[ 1.49826895  0.15904411]] 0.126375798478
X, y_new [[ 1.4457265   0.05323263]] 0.120732734183
X, y_new [[ 0.90061135  0.17293658]] 0.0830011990112
X, y_new [[ 0.95990536  0.5       ]] 0.0077396121276
X, y_new [[ 1.03730398  0.36072843]] 0.0117329057686
X, y_new [[ 1.13218597  0.5       ]] 0.00503855056315


In [78]:
dm

array([ 0.31023548,  0.04834544,  0.04834544,  0.04834544,  0.04834544,
        0.04834544,  0.04453188,  0.04453188,  0.04453188,  0.04453188,
        0.04453188,  0.04453188,  0.04453188])

In [80]:
ds

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [79]:
dm1

array([ 0.07660635,  0.07660635,  0.07660635,  0.07660635,  0.07660635,
        0.07184069,  0.0538892 ,  0.0538892 ,  0.0538892 ,  0.0538892 ,
        0.0538892 ])

In [67]:
ds1

array([ 0.07997355,  0.05063478,  0.03264229,  0.01790617,  0.01145286,
        0.0099168 ,  0.0075042 ,  0.00666383,  0.0043728 ,  0.00400936,
        0.00394636,  0.00386221,  0.00379728,  0.00380627,  0.00380807,
        0.00377238,  0.00376819,  0.00377534,  0.00376572,  0.00377217,
        0.00376286,  0.00375569,  0.00375971,  0.00375328,  0.00375052,
        0.00377452,  0.00377452,  0.00377452,  0.00377424,  0.00376966,
        0.00376568,  0.00376568,  0.00376568,  0.00376568,  0.00376568,
        0.00376568,  0.00376568,  0.00376568,  0.00376568])

In [46]:
################## Random Search #############
random.seed(datetime.now())

X = np.zeros((1,2))
RS_result =[]
RS_time =[]
# total_trial = 10
# mass_max = 0.3
# friction_max = 0.4

max_time = unit_time*number_of_samples
print 'max_time: ', max_time

for count in xrange(1,total_trial+1):
    print "starting trial", count, datetime.now()
    best_y = 10000
    best_f = 1
    best_m = 1
    result = []
    
    start_time = time.time()
    elapsed_time = time.time() - start_time

    while elapsed_time < max_time:
        m = (mass_max-mass_min)*random.random() + mass_min
        f = friction_max*random.random()
        X[0][0] = m
        X[0][1] = f

        y = myf(X)
        elapsed_time = time.time() - start_time

        if y < best_y:
            best_y = y
            best_m = m
            best_f = f

        result.append([m,f,y,best_m,best_f,best_y])
    print "optima!!!!!!!!!!!!!!!!: ", best_m,best_f,best_y, elapsed_time
    
    RS_result.append(result)
    RS_time.append(elapsed_time)
    
now = datetime.now()
save_data_filename = 'RS_result_'+str(now)
with open(save_data_filename, 'wb') as f:
    pickle.dump([RS_result,RS_time], f)
    
print "Finished ",count, " trials!" 

max_number = 0
for i in xrange(0,total_trial):  
#     print "current length:" , len(RS_result[i])
    if max_number <  len(RS_result[i]):
        max_number =  len(RS_result[i])
        print "max_number:", max_number

data1 = np.empty([total_trial,max_number])
for i in xrange(0,total_trial):  
    print len(RS_result[i])
    for j in xrange(0,len(RS_result[i])):        
        data1[i][j] = RS_result[i][j][5]
    for j in xrange(len(RS_result[i]), max_number):
        data1[i][j] = data1[i][j-1]
        
dm1 = np.mean(data1, axis=0)
ds1 = np.std(data1, axis=0)

max_time:  60
starting trial 1 2017-01-26 21:07:58.217197
X, y_new [[ 0.95039919  0.24111912]] 0.0448742446642
X, y_new [[ 0.80202765  0.12768594]] 0.119426565329
X, y_new [[ 0.82816154  0.18558173]] 0.0564612990328
X, y_new [[ 0.66264408  0.19022675]] 0.0575243481302
X, y_new [[ 0.77032694  0.19483015]] 0.0579106904486
X, y_new [[ 0.99374847  0.15720174]] 0.0832113414117
X, y_new [[ 0.50036968  0.22178242]] 0.0483118270109
X, y_new [[ 0.81310477  0.2452145 ]] 0.0466126315745
X, y_new [[ 0.76364735  0.07997752]] 0.209559709828
X, y_new [[ 0.8901306  0.0547301]] 0.264833862299
X, y_new [[ 0.53713281  0.04178874]] 0.277650246418
X, y_new [[ 0.92174481  0.24048177]] 0.045856627416
X, y_new [[ 0.99149697  0.21546909]] 0.0504945557773
X, y_new [[ 0.65385865  0.00229672]] 0.300846084976
X, y_new [[ 0.77924682  0.10560683]] 0.160873056244
X, y_new [[ 0.87805686  0.06868645]] 0.243134139723
X, y_new [[ 0.63991202  0.16910023]] 0.075050614476
X, y_new [[ 0.99103624  0.29496669]] 0.0438885044352

In [44]:
import pickle

now = datetime.now()
save_data_filename = 'RS_result_'+str(now)
with open(save_data_filename, 'wb') as f:
    pickle.dump([RS_result,RS_time], f)
    
print "Finished ",count, " trials!" 

max_number = 0
for i in xrange(0,total_trial):  
#     print "current length:" , len(RS_result[i])
    if max_number <  len(RS_result[i]):
        max_number =  len(RS_result[i])
        print "max_number:", max_number

data1 = np.empty([total_trial,max_number])
for i in xrange(0,total_trial):  
    print len(RS_result[i])
    for j in xrange(0,len(RS_result[i])):        
        data1[i][j] = RS_result[i][j][5]
    for j in xrange(len(RS_result[i]), max_number):
        data1[i][j] = data1[i][j-1]
        
dm1 = np.mean(data1, axis=0)
ds1 = np.std(data1, axis=0)

Finished  1  trials!
max_number: 37
37


In [41]:
import dill

now = datetime.now()

save_data_filename = 'BO_result_'+str(now)
with open(save_data_filename, 'wb') as f:
    dill.dump(BO_result, f)
print "Finished ",count, " trials!"

max_number = 0
for i in xrange(0,total_trial):  
#     print "current length:" , len(RS_result[i])
    if max_number <  len(BO_result[i].Y_best):
        max_number =  len(BO_result[i].Y_best)
        print "max_number:", max_number

data = np.empty([total_trial,max_number])
for i in xrange(0,total_trial):  
    print len(BO_result[i].Y_best)
    for j in xrange(0,len(BO_result[i].Y_best)):        
        data[i][j] = BO_result[i].Y_best[j]
    for j in xrange(len(BO_result[i].Y_best), max_number):
        data[i][j] = data[i][j-1]
dm = np.mean(data, axis=0)
ds = np.std(data, axis=0)

Finished  0  trials!
max_number: 19
19


In [61]:
myBopt.x_opt

array([ 0.59490441,  0.84025508])

In [38]:
BO_result[0].X_

In [40]:
BO_result[0].Y_best/20

array([ 0.0311558 ,  0.03006745,  0.02305226,  0.02305226,  0.02305226,
        0.02305226,  0.02303883,  0.02157928,  0.02151863,  0.02151863,
        0.02151863,  0.02151863,  0.02151863,  0.02151863,  0.02151863,
        0.02151863,  0.02033836,  0.02021556,  0.02021556,  0.02021556,
        0.02021556,  0.02021556,  0.02021556,  0.01982517,  0.01982517,
        0.01982517,  0.01982517,  0.01982517,  0.01973463,  0.01973463,
        0.01973463,  0.01973463,  0.01973463,  0.01926107,  0.01879421,
        0.01879421,  0.01879421,  0.01879421,  0.01879421,  0.01879421,
        0.01879421,  0.01879421,  0.01879421,  0.01879421,  0.01879421,
        0.01879421,  0.01879421,  0.01879421,  0.01879421,  0.01879421,
        0.01879421,  0.01879421,  0.01879421,  0.01879421,  0.01879421,
        0.01879421,  0.01879421,  0.01879421,  0.01879421,  0.01879421,
        0.01879421,  0.01879421,  0.01879421,  0.01879421,  0.01879421,
        0.01879421,  0.01879421,  0.01879421,  0.01879421,  0.01

In [ ]:
       print 'input: ', X[0][0], X[0][1], X_obj[0], X_obj[1], x_quat['w'], x_quat['x'], \
                         x_quat['y'], x_quat['z'], X_tip[0], X_tip[1], Y_tip[0], Y_tip[1], t
